In [32]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import time
from pyfm import pylibfm
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import gc

In [2]:
pd.set_option("display.max_columns", 1000)


In [3]:
log = pd.read_csv("~/environment/log.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")
users = pd.read_csv("~/environment/users.tsv", sep="\t")


In [4]:
print(log.shape)
log.head()

(1452089, 7)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN


In [5]:
log.loc[:,"user_id_str"] = log["user_id"].apply(lambda x:str(x))
log.loc[:,"event_id_str"] = log["event_id"].apply(lambda x:str(x))
log.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN,1,125296
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN,1,125128


In [6]:
data1 = pd.merge(log,events, on="event_id")
data = pd.merge(data1, users, on="user_id")
data = data.rename(columns = {"prefecture_x": "event_prefercture", "prefecture_y":"user_prefecture"})
data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefercture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26


In [7]:
data.loc[:,"event_date"] = pd.to_datetime(data["event_start_at"])
data.loc[:,"event_month"] = data.loc[:,"event_date"].apply(lambda x:str(x.month))
data.loc[:,"event_day"] = data.loc[:,"event_date"].apply(lambda x:str(x.day))
data.loc[:,"event_weekday"] = data.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
data.loc[:,"event_hour"] = data.loc[:,"event_date"].apply(lambda x:str(x.hour))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefercture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_month,event_day,event_weekday,event_hour
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26,2015-03-22 14:00:00,3,22,6,14
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,6,11,6,10
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,6,11,6,10
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,6,11,6,10
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,6,11,6,10


In [8]:
data.loc[:,"user_date"] = pd.to_datetime(data["created_on"])
data.loc[:,"user_year"] = data.loc[:,"user_date"].apply(lambda x:str(x.year))
data.loc[:,"user_month"] = data.loc[:,"user_date"].apply(lambda x:str(x.month))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefercture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_month,event_day,event_weekday,event_hour,user_date,user_year,user_month
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26,2015-03-22 14:00:00,3,22,6,14,2012-06-26,2012,6
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,6,11,6,10,2012-06-26,2012,6
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,6,11,6,10,2012-06-26,2012,6
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,6,11,6,10,2012-06-26,2012,6
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,6,11,6,10,2012-06-26,2012,6


In [9]:
features = ["female_age_lower","female_age_upper","male_age_lower","male_age_upper","event_prefercture","female_price","male_price","age", "gender", "user_prefecture", "event_month",
           "event_day","event_weekday", "event_hour", "user_year", "user_month"]
target = ["action_type"]

categorical = ["event_prefercture", "gender", "user_prefecture", "event_month","event_day","event_weekday", "event_hour", "user_year", "user_month"]



In [10]:
X = data[features]
print(X.shape)
X.head()

(1450405, 16)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefercture,female_price,male_price,age,gender,user_prefecture,event_month,event_day,event_weekday,event_hour,user_year,user_month
0,27,39.0,27,39.0,東京都,4000.0,6500.0,34,女性,静岡県,3,22,6,14,2012,6
1,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,6,11,6,10,2012,6
2,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,6,11,6,10,2012,6
3,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,6,11,6,10,2012,6
4,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,6,11,6,10,2012,6


In [29]:
#categoricalな特徴をダミー化する
#各特徴に専用の関数を作る

def month_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    category = np.array([1,2,3,4,5,6,7,8,9,10,11,12]).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    return onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
def month_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    category = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,34,35,36,37,38,39,30,31]).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    return onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()

def year_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    category = np.array([2012,2013,2014,2015,2016,2017]).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    return onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()

def weekday_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    category = np.array([0,1,2,3,4,5,6]).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    return onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()

In [80]:
def gender_dummying(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["女性","男性"]
    label_encoder.fit(labels)
    le = label_encoder.transform(x)
    
    #数値をonehot vectorへ
    
    onehot_encoder = OneHotEncoder()
    category = np.array(np.arange(len(labels)).reshape(-1,1))
    onehot_encoder.fit(category)
    
    return onehot_encoder.transform(np.array([le]).reshape(-1,1)).toarray()

def prefecture_dummying(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["北海道","青森県","岩手県","宮城県","秋田県", 
"山形県", "福島県", "茨城県","栃木県","群馬県", 
"埼玉県","千葉県","東京都","神奈川県","新潟県", 
"富山県","石川県","福井県","山梨県","長野県", 
"岐阜県","静岡県","愛知県","三重県","滋賀県", 
"京都府","大阪府","兵庫県","奈良県","和歌山県", 
"鳥取県","島根県","岡山県","広島県","山口県", 
"徳島県","香川県","愛媛県","高知県","福岡県", 
"佐賀県","長崎県","熊本県","大分県","宮崎県", 
"鹿児島県","沖縄県"]
    label_encoder.fit(labels)
    le =label_encoder.transform(x)
    
    #数値をonehot vectorへ
    
    onehot_encoder = OneHotEncoder()
    category = np.array(np.arrange(len(labels))).reshape(-1,1)
    onehot_encoder.fit(category)
    
    return onehot_encoder.transform(np.array([le]).reshape(-1,1)).toarray()

In [81]:
 #文字列をラベリング
label_encoder = LabelEncoder()
labels = ["女性","男性"]
label_encoder.fit(labels)
    
#数値をonehot vectorへ
le = label_encoder.transform(["女性"])
onehot_encoder = OneHotEncoder()
category = np.array(np.arange(len(labels)).reshape(-1,1))
onehot_encoder.fit(category)

onehot_encoder.transform(np.array([le]).reshape(-1,1)).toarray()

array([[ 1.,  0.]])

In [82]:
gender_dummying(["女性"])

array([[ 1.,  0.]])

In [28]:
month_dummying(X["event_month"].values)

array([[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [48]:
X = pd.get_dummies(X)
print(X.shape)


(1450405, 192)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,female_price,male_price,age,event_prefercture_三重県,event_prefercture_京都府,event_prefercture_佐賀県,event_prefercture_兵庫県,event_prefercture_北海道,event_prefercture_千葉県,event_prefercture_和歌山県,event_prefercture_埼玉県,event_prefercture_大分県,event_prefercture_大阪府,event_prefercture_奈良県,event_prefercture_宮城県,event_prefercture_宮崎県,event_prefercture_富山県,event_prefercture_山口県,event_prefercture_山形県,event_prefercture_山梨県,event_prefercture_岐阜県,event_prefercture_岡山県,event_prefercture_岩手県,event_prefercture_島根県,event_prefercture_広島県,event_prefercture_徳島県,event_prefercture_愛媛県,event_prefercture_愛知県,event_prefercture_新潟県,event_prefercture_東京都,event_prefercture_栃木県,event_prefercture_沖縄県,event_prefercture_滋賀県,event_prefercture_熊本県,event_prefercture_石川県,event_prefercture_神奈川県,event_prefercture_福井県,event_prefercture_福岡県,event_prefercture_福島県,event_prefercture_秋田県,event_prefercture_群馬県,event_prefercture_茨城県,event_prefercture_長崎県,event_prefercture_長野県,event_prefercture_青森県,event_prefercture_静岡県,...,event_day_5,event_day_6,event_day_7,event_day_8,event_day_9,event_weekday_0,event_weekday_1,event_weekday_2,event_weekday_3,event_weekday_4,event_weekday_5,event_weekday_6,event_hour_0,event_hour_10,event_hour_11,event_hour_12,event_hour_13,event_hour_14,event_hour_15,event_hour_16,event_hour_17,event_hour_18,event_hour_19,event_hour_20,event_hour_21,event_hour_22,event_hour_23,event_hour_5,event_hour_6,event_hour_7,event_hour_8,event_hour_9,user_year_2012,user_year_2013,user_year_2014,user_year_2015,user_year_2016,user_year_2017,user_month_1,user_month_10,user_month_11,user_month_12,user_month_2,user_month_3,user_month_4,user_month_5,user_month_6,user_month_7,user_month_8,user_month_9
0,27,39.0,27,39.0,4000.0,6500.0,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,24,36.0,26,38.0,2000.0,6000.0,34,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,24,36.0,26,38.0,2000.0,6000.0,34,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,24,36.0,26,38.0,2000.0,6000.0,34,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,24,36.0,26,38.0,2000.0,6000.0,34,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [65]:
X.head()

,female_age_lower,female_age_upper,male_age_lower,male_age_upper,female_price,male_price,age,event_prefercture_三重県,event_prefercture_京都府,event_prefercture_佐賀県,event_prefercture_兵庫県,event_prefercture_北海道,event_prefercture_千葉県,event_prefercture_和歌山県,event_prefercture_埼玉県,event_prefercture_大分県,event_prefercture_大阪府,event_prefercture_奈良県,event_prefercture_宮城県,event_prefercture_宮崎県,event_prefercture_富山県,event_prefercture_山口県,event_prefercture_山形県,event_prefercture_山梨県,event_prefercture_岐阜県,event_prefercture_岡山県,event_prefercture_岩手県,event_prefercture_島根県,event_prefercture_広島県,event_prefercture_徳島県,event_prefercture_愛媛県,event_prefercture_愛知県,event_prefercture_新潟県,event_prefercture_東京都,event_prefercture_栃木県,event_prefercture_沖縄県,event_prefercture_滋賀県,event_prefercture_熊本県,event_prefercture_石川県,event_prefercture_神奈川県,event_prefercture_福井県,event_prefercture_福岡県,event_prefercture_福島県,event_prefercture_秋田県,event_prefercture_群馬県,event_prefercture_茨城県,event_prefercture_長崎県,event_prefercture_長野県,event_prefercture_青森県,event_prefercture_静岡県,event_prefercture_香川県,event_prefercture_高知県,event_prefercture_鳥取県,event_prefercture_鹿児島県,gender_女性,gender_男性,user_prefecture_その他（海外等）,user_prefecture_三重県,user_prefecture_京都府,user_prefecture_佐賀県,user_prefecture_兵庫県,user_prefecture_北海道,user_prefecture_千葉県,user_prefecture_和歌山県,user_prefecture_埼玉県,user_prefecture_大分県,user_prefecture_大阪府,user_prefecture_奈良県,user_prefecture_宮城県,user_prefecture_宮崎県,user_prefecture_富山県,user_prefecture_山口県,user_prefecture_山形県,user_prefecture_山梨県,user_prefecture_岐阜県,user_prefecture_岡山県,user_prefecture_岩手県,user_prefecture_島根県,user_prefecture_広島県,user_prefecture_徳島県,user_prefecture_愛媛県,user_prefecture_愛知県,user_prefecture_新潟県,user_prefecture_東京都,user_prefecture_栃木県,user_prefecture_沖縄県,user_prefecture_滋賀県,user_prefecture_熊本県,user_prefecture_石川県,user_prefecture_神奈川県,user_prefecture_福井県,user_prefecture_福岡県,user_prefecture_福島県,user_prefecture_秋田県,user_prefecture_群馬県,user_prefecture_茨城県,user_prefecture_長崎県,user_prefecture_長野県,user_prefecture_青森県,user_prefecture_静岡県,user_prefecture_香川県,user_prefecture_高知県,user_prefecture_鳥取県,user_prefecture_鹿児島県,event_month_1,event_month_10,event_month_11,event_month_12,event_month_2,event_month_3,event_month_4,event_month_5,event_month_6,event_month_7,event_month_8,event_month_9,event_day_1,event_day_10,event_day_11,event_day_12,event_day_13,event_day_14,event_day_15,event_day_16,event_day_17,event_day_18,event_day_19,event_day_2,event_day_20,event_day_21,event_day_22,event_day_23,event_day_24,event_day_25,event_day_26,event_day_27,event_day_28,event_day_29,event_day_3,event_day_30,event_day_31,event_day_4,event_day_5,event_day_6,event_day_7,event_day_8,event_day_9,event_weekday_0,event_weekday_1,event_weekday_2,event_weekday_3,event_weekday_4,event_weekday_5,event_weekday_6,event_hour_0,event_hour_10,event_hour_11,event_hour_12,event_hour_13,event_hour_14,event_hour_15,event_hour_16,event_hour_17,event_hour_18,event_hour_19,event_hour_20,event_hour_21,event_hour_22,event_hour_23,event_hour_5,event_hour_6,event_hour_7,event_hour_8,event_hour_9,user_year_2012,user_year_2013,user_year_2014,user_year_2015,user_year_2016,user_year_2017,user_month_1,user_month_10,user_month_11,user_month_12,user_month_2,user_month_3,user_month_4,user_month_5,user_month_6,user_month_7,user_month_8,user_month_9
0,27,39.0,27,39.0,4000.0,6500.0,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,24,36.0,26,38.0,2000.0,6000.0,34,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

In [50]:
y = data.loc[:,target]
print(y.shape)
y.head()

(1450405, 1)


,action_type
0,3
1,1
2,1
3,1
4,1


In [51]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [52]:
y_train = np.array(y_train).reshape(-1)
y_train = y_train.astype(np.float64)
print(y_train.shape)
print(y_train.dtype)

(1160324,)
float64


In [53]:
y_valid = np.array(y_valid).reshape(-1)
y_valid = y_valid.astype(np.float64)
print(y_valid.shape)
print(y_valid.dtype)

(290081,)
float64


In [54]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

In [55]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': {'multi_logloss'},
        'num_class': 4,
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 10000,
        'verbose': 0
}


In [56]:
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=50,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)


/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 1.04787
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 1.01265
[3]	valid_0's multi_logloss: 0.98269
[4]	valid_0's multi_logloss: 0.957126
[5]	valid_0's multi_logloss: 0.935234
[6]	valid_0's multi_logloss: 0.91625
[7]	valid_0's multi_logloss: 0.899624
[8]	valid_0's multi_logloss: 0.885248
[9]	valid_0's multi_logloss: 0.872624
[10]	valid_0's multi_logloss: 0.861574
[11]	valid_0's multi_logloss: 0.851908
[12]	valid_0's multi_logloss: 0.84339
[13]	valid_0's multi_logloss: 0.835794
[14]	valid_0's multi_logloss: 0.82909
[15]	valid_0's multi_logloss: 0.823161
[16]	valid_0's multi_logloss: 0.817796
[17]	valid_0's multi_logloss: 0.813046
[18]	valid_0's multi_logloss: 0.808835
[19]	valid_0's multi_logloss: 0.805088
[20]	valid_0's multi_logloss: 0.801632
[21]	valid_0's multi_logloss: 0.798527
[22]	valid_0's multi_logloss: 0.795692
[23]	valid_0's multi_logloss: 0.793154
[24]	valid_0's multi_logloss: 0.790829
[25]	valid_0's mu

[208]	valid_0's multi_logloss: 0.710716
[209]	valid_0's multi_logloss: 0.710595
[210]	valid_0's multi_logloss: 0.710422
[211]	valid_0's multi_logloss: 0.710322
[212]	valid_0's multi_logloss: 0.710243
[213]	valid_0's multi_logloss: 0.710121
[214]	valid_0's multi_logloss: 0.710031
[215]	valid_0's multi_logloss: 0.709944
[216]	valid_0's multi_logloss: 0.709809
[217]	valid_0's multi_logloss: 0.709679
[218]	valid_0's multi_logloss: 0.709617
[219]	valid_0's multi_logloss: 0.709442
[220]	valid_0's multi_logloss: 0.709374
[221]	valid_0's multi_logloss: 0.709315
[222]	valid_0's multi_logloss: 0.709166
[223]	valid_0's multi_logloss: 0.709043
[224]	valid_0's multi_logloss: 0.708853
[225]	valid_0's multi_logloss: 0.708734
[226]	valid_0's multi_logloss: 0.708604
[227]	valid_0's multi_logloss: 0.708467
[228]	valid_0's multi_logloss: 0.708246
[229]	valid_0's multi_logloss: 0.708119
[230]	valid_0's multi_logloss: 0.708013
[231]	valid_0's multi_logloss: 0.707926
[232]	valid_0's multi_logloss: 0.707798


[414]	valid_0's multi_logloss: 0.690328
[415]	valid_0's multi_logloss: 0.690293
[416]	valid_0's multi_logloss: 0.690225
[417]	valid_0's multi_logloss: 0.690187
[418]	valid_0's multi_logloss: 0.690111
[419]	valid_0's multi_logloss: 0.690067
[420]	valid_0's multi_logloss: 0.690039
[421]	valid_0's multi_logloss: 0.689999
[422]	valid_0's multi_logloss: 0.689931
[423]	valid_0's multi_logloss: 0.689827
[424]	valid_0's multi_logloss: 0.68974
[425]	valid_0's multi_logloss: 0.689672
[426]	valid_0's multi_logloss: 0.689552
[427]	valid_0's multi_logloss: 0.689476
[428]	valid_0's multi_logloss: 0.689416
[429]	valid_0's multi_logloss: 0.6893
[430]	valid_0's multi_logloss: 0.689197
[431]	valid_0's multi_logloss: 0.689073
[432]	valid_0's multi_logloss: 0.689025
[433]	valid_0's multi_logloss: 0.688993
[434]	valid_0's multi_logloss: 0.688847
[435]	valid_0's multi_logloss: 0.688779
[436]	valid_0's multi_logloss: 0.6887
[437]	valid_0's multi_logloss: 0.688611
[438]	valid_0's multi_logloss: 0.688581
[439]

[620]	valid_0's multi_logloss: 0.677966
[621]	valid_0's multi_logloss: 0.677921
[622]	valid_0's multi_logloss: 0.677834
[623]	valid_0's multi_logloss: 0.677732
[624]	valid_0's multi_logloss: 0.677644
[625]	valid_0's multi_logloss: 0.677509
[626]	valid_0's multi_logloss: 0.677543
[627]	valid_0's multi_logloss: 0.677503
[628]	valid_0's multi_logloss: 0.677374
[629]	valid_0's multi_logloss: 0.677248
[630]	valid_0's multi_logloss: 0.677265
[631]	valid_0's multi_logloss: 0.67812
[632]	valid_0's multi_logloss: 0.677501
[633]	valid_0's multi_logloss: 0.677265
[634]	valid_0's multi_logloss: 0.677164
[635]	valid_0's multi_logloss: 0.67712
[636]	valid_0's multi_logloss: 0.677076
[637]	valid_0's multi_logloss: 0.677039
[638]	valid_0's multi_logloss: 0.677008
[639]	valid_0's multi_logloss: 0.676987
[640]	valid_0's multi_logloss: 0.67689
[641]	valid_0's multi_logloss: 0.676877
[642]	valid_0's multi_logloss: 0.67687
[643]	valid_0's multi_logloss: 0.676777
[644]	valid_0's multi_logloss: 0.676729
[645

[826]	valid_0's multi_logloss: 0.666696
[827]	valid_0's multi_logloss: 0.666538
[828]	valid_0's multi_logloss: 0.666421
[829]	valid_0's multi_logloss: 0.666354
[830]	valid_0's multi_logloss: 0.666298
[831]	valid_0's multi_logloss: 0.666263
[832]	valid_0's multi_logloss: 0.666238
[833]	valid_0's multi_logloss: 0.666201
[834]	valid_0's multi_logloss: 0.666162
[835]	valid_0's multi_logloss: 0.666101
[836]	valid_0's multi_logloss: 0.666049
[837]	valid_0's multi_logloss: 0.665991
[838]	valid_0's multi_logloss: 0.66598
[839]	valid_0's multi_logloss: 0.666073
[840]	valid_0's multi_logloss: 0.666054
[841]	valid_0's multi_logloss: 0.665956
[842]	valid_0's multi_logloss: 0.665939
[843]	valid_0's multi_logloss: 0.665906
[844]	valid_0's multi_logloss: 0.665814
[845]	valid_0's multi_logloss: 0.665744
[846]	valid_0's multi_logloss: 0.665686
[847]	valid_0's multi_logloss: 0.665647
[848]	valid_0's multi_logloss: 0.665631
[849]	valid_0's multi_logloss: 0.66551
[850]	valid_0's multi_logloss: 0.665441
[8

[1031]	valid_0's multi_logloss: 0.657385
[1032]	valid_0's multi_logloss: 0.657371
[1033]	valid_0's multi_logloss: 0.657342
[1034]	valid_0's multi_logloss: 0.657299
[1035]	valid_0's multi_logloss: 0.65725
[1036]	valid_0's multi_logloss: 0.657208
[1037]	valid_0's multi_logloss: 0.657168
[1038]	valid_0's multi_logloss: 0.657145
[1039]	valid_0's multi_logloss: 0.657033
[1040]	valid_0's multi_logloss: 0.656924
[1041]	valid_0's multi_logloss: 0.656827
[1042]	valid_0's multi_logloss: 0.656763
[1043]	valid_0's multi_logloss: 0.656733
[1044]	valid_0's multi_logloss: 0.656672
[1045]	valid_0's multi_logloss: 0.656647
[1046]	valid_0's multi_logloss: 0.656593
[1047]	valid_0's multi_logloss: 0.656554
[1048]	valid_0's multi_logloss: 0.656517
[1049]	valid_0's multi_logloss: 0.656445
[1050]	valid_0's multi_logloss: 0.656336
[1051]	valid_0's multi_logloss: 0.656224
[1052]	valid_0's multi_logloss: 0.656183
[1053]	valid_0's multi_logloss: 0.656074
[1054]	valid_0's multi_logloss: 0.656013
[1055]	valid_0's 

[1232]	valid_0's multi_logloss: 0.64882
[1233]	valid_0's multi_logloss: 0.648745
[1234]	valid_0's multi_logloss: 0.648713
[1235]	valid_0's multi_logloss: 0.648681
[1236]	valid_0's multi_logloss: 0.648653
[1237]	valid_0's multi_logloss: 0.648602
[1238]	valid_0's multi_logloss: 0.648558
[1239]	valid_0's multi_logloss: 0.648536
[1240]	valid_0's multi_logloss: 0.648526
[1241]	valid_0's multi_logloss: 0.6485
[1242]	valid_0's multi_logloss: 0.648473
[1243]	valid_0's multi_logloss: 0.648472
[1244]	valid_0's multi_logloss: 0.64845
[1245]	valid_0's multi_logloss: 0.648443
[1246]	valid_0's multi_logloss: 0.648411
[1247]	valid_0's multi_logloss: 0.648398
[1248]	valid_0's multi_logloss: 0.648361
[1249]	valid_0's multi_logloss: 0.64833
[1250]	valid_0's multi_logloss: 0.648269
[1251]	valid_0's multi_logloss: 0.648232
[1252]	valid_0's multi_logloss: 0.648205
[1253]	valid_0's multi_logloss: 0.648198
[1254]	valid_0's multi_logloss: 0.648182
[1255]	valid_0's multi_logloss: 0.64814
[1256]	valid_0's multi

[1433]	valid_0's multi_logloss: 0.641744
[1434]	valid_0's multi_logloss: 0.641726
[1435]	valid_0's multi_logloss: 0.641669
[1436]	valid_0's multi_logloss: 0.641616
[1437]	valid_0's multi_logloss: 0.641597
[1438]	valid_0's multi_logloss: 0.641579
[1439]	valid_0's multi_logloss: 0.641548
[1440]	valid_0's multi_logloss: 0.641522
[1441]	valid_0's multi_logloss: 0.641512
[1442]	valid_0's multi_logloss: 0.641498
[1443]	valid_0's multi_logloss: 0.641487
[1444]	valid_0's multi_logloss: 0.641479
[1445]	valid_0's multi_logloss: 0.641471
[1446]	valid_0's multi_logloss: 0.64146
[1447]	valid_0's multi_logloss: 0.641456
[1448]	valid_0's multi_logloss: 0.641444
[1449]	valid_0's multi_logloss: 0.641435
[1450]	valid_0's multi_logloss: 0.641387
[1451]	valid_0's multi_logloss: 0.641372
[1452]	valid_0's multi_logloss: 0.641367
[1453]	valid_0's multi_logloss: 0.641362
[1454]	valid_0's multi_logloss: 0.64136
[1455]	valid_0's multi_logloss: 0.641345
[1456]	valid_0's multi_logloss: 0.641335
[1457]	valid_0's m

In [60]:
y_pred = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
y_pred = np.argmax(y_pred, axis=1)

In [61]:
y_pred

array([2, 3, 2, ..., 1, 1, 1])

In [63]:
pd.Series(y_pred).value_counts()

1    220086
2     47171
3     22824
dtype: int64

In [88]:
test = pd.read_csv("~/environment/test.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")

In [89]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


In [28]:
user_list = test["user_id"]
event_list = events["event_id"]

NameError: name 'test' is not defined

In [94]:
events.loc[:,"user_id"] = 1
user_data = pd.merge(events, users, on = "user_id")
data = data.rename(columns = {"prefecture_x": "event_prefercture", "prefecture_y":"user_prefecture"})

data.loc[:,"event_date"] = pd.to_datetime(data["event_start_at"])
data.loc[:,"event_month"] = data.loc[:,"event_date"].apply(lambda x:str(x.month))
data.loc[:,"event_day"] = data.loc[:,"event_date"].apply(lambda x:str(x.day))
data.loc[:,"event_weekday"] = data.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
data.loc[:,"event_hour"] = data.loc[:,"event_date"].apply(lambda x:str(x.hour))

data.loc[:,"user_date"] = pd.to_datetime(data["created_on"])
data.loc[:,"user_year"] = data.loc[:,"user_date"].apply(lambda x:str(x.year))
data.loc[:,"user_month"] = data.loc[:,"user_date"].apply(lambda x:str(x.month))

user_data.head()

,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,user_id,age,gender,prefecture_y,created_on
0,1,20,99.0,25,44.0,2015-03-08 18:00:00,宮城県,NaN,1500.0,5000.0,NaN,1,34,女性,静岡県,2012-06-26
1,2,30,99.0,30,49.0,2015-03-01 15:00:00,富山県,NaN,1000.0,6500.0,NaN,1,34,女性,静岡県,2012-06-26
2,3,22,99.0,24,39.0,2015-03-07 19:00:00,富山県,NaN,1000.0,6000.0,NaN,1,34,女性,静岡県,2012-06-26
3,4,20,99.0,25,44.0,2015-03-08 18:00:00,新潟県,NaN,1000.0,6000.0,NaN,1,34,女性,静岡県,2012-06-26
4,5,20,99.0,20,39.0,2015-03-14 19:00:00,新潟県,NaN,1000.0,6000.0,NaN,1,34,女性,静岡県,2012-06-26


In [92]:
user_X = user_data[features]
user_X = pd.get_dummies(user_X)
print(user_X.shape)


KeyError: "['event_prefercture' 'user_prefecture' 'event_month' 'event_day'\n 'event_weekday' 'event_hour' 'user_year' 'user_month'] not in index"

In [15]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

le = LabelEncoder()
oe = OneHotEncoder()
en = le.fit_transform(['orange','orange','apple','banana','apple'])
print(oe.fit_transform(en.reshape(-1, 1)).toarray())

[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]]


In [16]:
en

array([2, 2, 0, 1, 0])

In [17]:
oe.fit_transform(en.reshape(-1, 1))

<5x3 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [19]:
en = le.fit(['orange','orange','apple','banana','apple'])
en

LabelEncoder()

In [52]:
le = LabelEncoder()
labels = ['orange','apple','banana']
le.fit(labels)


LabelEncoder()

In [53]:
le.transform(["orange"])

array([2])

In [12]:
oe = OneHotEncoder()
labels = np.array([0,1,2]).reshape(-1,1)
oen = oe.fit(labels)

In [17]:
d = np.array([0]).reshape(-1,1) 
oen.transform(d).toarray()

array([[ 1.,  0.,  0.]])